In [1]:
!pip install llmware

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 MB 3.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 4.0 MB/s eta 0:00:0000:0100:010m


In [2]:
!pip install fastapi nest-asyncio pyngrok uvicorn 

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 3.6 MB/s eta 0:00:00


In [ ]:
from llmware.models import ModelCatalog

model = ModelCatalog().load_model("bartowski/Meta-Llama-3-8B-Instruct-GGUF")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:832: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Meta-Llama-3-8B-Instruct-Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware

app = FastAPI()
app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'], 
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'], 
)

In [ ]:
from typing import Union
from pydantic import BaseModel


def get_response(prompt:str, context:Union[None, str]=""):
    response = model.inference(prompt, add_context=context)
    return response


class ChatRequest(BaseModel):
    prompt: str
    context: Union[str, None] = None

@app.post("/chat")
async def chat_endpoint(chat_request: ChatRequest):
    try:
        result = get_response(chat_request.prompt, chat_request.context)
        
        return result
    except ValueError as e:
        raise HTTPException(status_code=500, detail=str(e))

In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Get your authtoken from https://dashboard.ngrok.com/get-started/your-authtoken
auth_token = "authapixxxxxxxxxxxxxxxxxxxxxxxx"

# Set the authtoken
ngrok.set_auth_token(auth_token)

# Connect to ngrok
ngrok_tunnel = ngrok.connect(8000, hostname='your-host-name.ngrok-free.app')


print('Public URL:', ngrok_tunnel.public_url)

# Apply nest_asyncio
nest_asyncio.apply()

# Run the uvicorn server
uvicorn.run(app, port=8000)